# ⚔️ Strategy Arena

Welcome to the Arena. Here we pit different trading strategies against each other to see who survives the market.

### Contenders:
1. **Buy & Hold**: The benchmark.
2. **Simple DCA**: Regular monthly investment.
3. **MA 200**: Classic trend following (Above=Buy, Below=Sell).
4. **David (Ladder)**: Blue Ladder Breakout/Breakdown.
5. **TQQQ DCA+**: Advanced cash management & profit taking.

---

In [71]:
import sys
import os
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output

# Add project root
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
# Reload module to ensure latest code (fixes Jupyter caching)
import importlib
import sys
if 'core.strategies' in sys.modules:
    importlib.reload(sys.modules['core.strategies'])

from core.strategies import BuyAndHold, SimpleDCA, MA200Strategy, DavidStrategy, TQQQ_DCA_Plus

# Cache
DATA_CACHE = {}

def get_data(ticker):
    if ticker not in DATA_CACHE:
        print(f"Fetching {ticker}...")
        # Get max history
        df = yf.download(ticker, period="max", interval="1d", progress=False)
        if isinstance(df.columns, pd.MultiIndex):
            try:
                df.columns = df.columns.droplevel(1)
            except:
                pass
        # Filter to reasonable start date if needed, or keep full history
        DATA_CACHE[ticker] = df
    return DATA_CACHE[ticker].copy()


In [72]:
def run_arena(ticker, initial_cash=100000, start_date=None, end_date=None):
    # Force reload to ensure latest strategy code
    import importlib
    import sys
    if 'core.strategies' in sys.modules:
        importlib.reload(sys.modules['core.strategies'])
        # Re-import after reload
        from core.strategies import BuyAndHold, SimpleDCA, MA200Strategy, DavidStrategy, TQQQ_DCA_Plus
    
    df = get_data(ticker)
    
    # Date filtering (optional)
    if start_date:
        df = df[df.index >= pd.to_datetime(start_date)]
    if end_date:
        df = df[df.index <= pd.to_datetime(end_date)]
    if df.empty:
        print("Error: No data in selected range.")
        return
    
    # Define Contenders
    strategies = [
        BuyAndHold(initial_cash),
        SimpleDCA(initial_cash, monthly_invest=1000),
        MA200Strategy(initial_cash),
        DavidStrategy(initial_cash),
        TQQQ_DCA_Plus(initial_cash)
    ]
    
    results = {}
    
    print(f"\nRunning simulations for {ticker}...")
    
    for strat in strategies:
        try:
            res_df = strat.run(df)
            
            # Metrics
            final_eq = res_df['Equity'].iloc[-1]
            ret = (final_eq - initial_cash) / initial_cash
            
            # Max Drawdown
            roll_max = res_df['Equity'].cummax()
            dd = (res_df['Equity'] - roll_max) / roll_max
            max_dd = dd.min()
            
            # Debug: Check DavidStrategy equity around May
            if strat.name == 'David (Ladder)':
                may_equity = res_df['Equity'][(res_df.index >= '2025-04-30') & (res_df.index <= '2025-05-02')]
                if len(may_equity) > 0:
                    print(f"\n🔍 {strat.name} May check:")
                    for date, eq in may_equity.items():
                        print(f"  {date.date()}: ${eq:,.2f}")
            
            results[strat.name] = {
                'Equity': res_df['Equity'],
                'Return': ret,
                'MaxDD': max_dd,
                'Final': final_eq
            }
        except Exception as e:
            print(f"Strategy {strat.name} failed: {e}")
            
    # --- VISUALIZATION ---
    
    # 1. Performance Table
    metrics = pd.DataFrame(results).T[['Return', 'MaxDD', 'Final']]
    metrics['Return'] = metrics['Return'].apply(lambda x: f"{x:.2%}")
    metrics['MaxDD'] = metrics['MaxDD'].apply(lambda x: f"{x:.2%}")
    metrics['Final'] = metrics['Final'].apply(lambda x: f"${x:,.0f}")
    
    print("\n🏆 FINAL SCOREBOARD")
    display(metrics.sort_values('Final', ascending=False))
    
    # 2. Chart
    plt.figure(figsize=(14, 8))
    # Ensure all Equity series use the same index (from the original df)
    common_index = df.index
    for name, data in results.items():
        equity_series = data['Equity']
        # Reindex to common_index, forward fill to handle any missing dates
        equity_aligned = equity_series.reindex(common_index, method='ffill')
        plt.plot(common_index, equity_aligned.values, label=name, linewidth=2)
        
    plt.title(f"Strategy Comparison: {ticker}")
    plt.yscale('log')
    plt.ylabel('Portfolio Value (Log Scale)')
    plt.legend()
    plt.grid(True, which="both", ls="-", alpha=0.2)
    plt.show()

In [73]:
# --- UI ---
w_ticker = widgets.Text(value='TQQQ', description='Ticker:', style={'description_width': 'initial'})

# Date Pickers (fallback to Text if DatePicker not available)
from datetime import date
default_start = date(2025, 1, 4)
default_end = date(2025, 12, 4)

try:
    w_start = widgets.DatePicker(description='Start Date:', value=default_start, disabled=False, style={'description_width': 'initial'})
    w_end = widgets.DatePicker(description='End Date:', value=default_end, disabled=False, style={'description_width': 'initial'})
except:
    # Fallback for older ipywidgets versions
    w_start = widgets.Text(value='2025-01-04', description='Start (YYYY-MM-DD):', style={'description_width': 'initial'})
    w_end = widgets.Text(value='2025-12-04', description='End (YYYY-MM-DD):', style={'description_width': 'initial'})

w_btn = widgets.Button(description='FIGHT! ⚔️', button_style='danger', icon='play')

out = widgets.Output()

def on_click(b):
    with out:
        from IPython.display import clear_output
        from datetime import date, datetime
        clear_output(wait=True)
        
        # Handle DatePicker or Text input
        def format_date(val):
            if val is None:
                return None
            if isinstance(val, (date, datetime, pd.Timestamp)):
                return val.strftime('%Y-%m-%d')
            elif isinstance(val, str):
                return val.strip() if val.strip() else None
            return None
        
        start = format_date(w_start.value)
        end = format_date(w_end.value)
        
        run_arena(w_ticker.value, start_date=start, end_date=end)

w_btn.on_click(on_click)

# Layout
display(widgets.VBox([
    widgets.HBox([w_ticker, w_btn]),
    widgets.HBox([w_start, w_end])
]))
display(out)

Output()